In [ ]:
# Dependencies
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time
import pandas as pd

In [ ]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

In [ ]:
# Retrieve page with the requests module
response = requests.get(url)

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')

In [ ]:
 # Examine the results, then determine element that contains sought info
print(soup.prettify())

In [ ]:
 # Print all paragraph texts
titles = soup.find_all('div', class_='content_title')
for article_title in titles:
    print(article_title.text)

In [ ]:
# find the latest title and paragraph then store in variables
news_title = soup.find("div", class_="content_title").text
news_p = soup.find("div", class_="rollover_description_inner").text

print(f"Title: {news_title}")
print(f"Paragraph: {news_p}")

In [ ]:
# Set the directory where chromedriver exists
executable_path = {'executable_path':'C:/bin/chromedriver.exe'} 
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)
time.sleep(1)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(1)
expand = browser.find_by_css('a.fancybox-expand')
expand.click()
time.sleep(1)

jpl_html = browser.html
jpl_soup = bs(jpl_html, 'html.parser')

img_relative = jpl_soup.find('img', class_='fancybox-image')['src']
image_path = f'https://www.jpl.nasa.gov{img_relative}'
print(image_path)

In [ ]:
# Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. 
# Save the tweet text for the weather report as a variable called mars_weather
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(mars_weather_url)
time.sleep(1)
mars_weather_html = browser.html
mars_weather_soup = bs(mars_weather_html, 'html.parser')

latest_tweet = mars_weather_soup.find('ol', class_='data-testid')
#mars_weather = latest_tweet.find('p', class_='tweet').text
print(mars_weather_url)

In [ ]:
# Visit the Mars Facts webpage here and 
#use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
mars_facts_url = 'https://space-facts.com/mars/'
browser.visit(mars_facts_url)
time.sleep(1)
mars_facts_html = browser.html
mars_facts_soup = bs(mars_facts_html, 'html.parser')

fact_table = mars_facts_soup.find('table', class_='tablepress tablepress-id-p-mars')
column1 = fact_table.find_all('td', class_='column-1')
column2 = fact_table.find_all('td', class_='column-2')

# I needed help with this-found an example online
facets = []
values = []

for row in column1:
    facet = row.text.strip()
    facets.append(facet)
    
for row in column2:
    value = row.text.strip()
    values.append(value)
    
mars_facts = pd.DataFrame({
    "Facet":facets,
    "Value":values
    })

mars_facts_html = mars_facts.to_html(header=False, index=False)
mars_facts